START OF THE FINAL PROJECT

All of the import statements

In [1]:
import pandas as pd
import transformers
from torch import torch_version
from torch import nn
import torch

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


1)LOAD THE DATASET

In [2]:
import pandas as pd
import string
import re 
# from config import CONFIG



class PREPROCESS:
    def __init__(self) -> None:
        df = pd.read_csv("output_csv_file.csv")
        df["prepText"] = df["Lyrics"].apply(self.cleanString)
        df = df[["Artist", "Title", "prepText","Label"]]
        # save the file 
        df.to_csv("preprocessed_songs_gram.csv", index=False)
    
    def cleanString(self,sentence):
        sentence = sentence.translate(str.maketrans("","", string.punctuation)).strip().lower()
        sentence = re.sub(r"https?://\s+", "", sentence)
        sentence = re.sub(r"\b\d+\b",  "", sentence)
        sentence = re.sub(r" +"," ",sentence).replace("\n", " ").replace("\r", "").replace("\r\n", "")
        sentence = re.sub("\s+", " ", sentence)
        return sentence    




In [39]:
PREPROCESS()


In [44]:
from sentence_transformers import SentenceTransformer

import os
import numpy as np
from tqdm import tqdm
# import faiss
class findEmbedding:
    def __init__(self) -> None:
        self.df = pd.read_csv("preprocessed_songs_gram.csv")
        self.model = SentenceTransformer('all-MiniLM-L6-v2')
        embeddings = self.computeEmbeddings()
        if not os.path.isdir("results"):
            os.mkdir("results")
        PATH = os.path.join("embedding.npy")
        with open( PATH, "wb") as fp:
            np.save(fp, embeddings)

    def computeEmbeddings(self):
        sentences = self.df["prepText"].values
        embeddings = []
        for sentence in tqdm(sentences):
            embedding = self.model.encode(sentence)
            embeddings.append(embedding)
        return np.array(embeddings)

In [45]:
findEmbedding()

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
100%|██████████| 773/773 [00:42<00:00, 18.14it/s]


In [8]:
import faiss
class recmmSystem:
    def __init__(self) -> None:
        self.model = SentenceTransformer('all-MiniLM-L6-v2')
        self.df = pd.read_csv("preprocessed_songs.csv")
        PATH = os.path.join( "embedding.npy")
        embeddings = np.load(PATH)
        dimension = embeddings.shape[1]
        quantize = faiss.IndexFlatL2(dimension)
        self.index = faiss.IndexIVFFlat(quantize, dimension, 50)

        if not self.index.is_trained:
            print(1)
            self.index.train(embeddings)
        
        if self.index.is_trained:
            print(0)
            self.index.add(embeddings)

        print("Total Number of embeddings index {}".format(self.index.ntotal))
    
    def nearestNeighbour(self, query, k):
        queryEmbeddings = self.model.encode([query])
        D, I = self.index.search(queryEmbeddings, k)
        artist_songs = dict()
        allRelatable = self.df.iloc[I[0]]
        for index, rows in allRelatable.iterrows():
            print(rows["artist"])
            artist_songs[rows["artist"]] = rows["song"]
        
        print("Query: ", query)
        print(artist_songs)
findEmbedding()
recmmSystem()

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
100%|██████████| 9/9 [00:00<00:00, 40.57it/s]


NameError: name 'faiss' is not defined

In [27]:
from torch import optim
from torch.utils.data import DataLoader, TensorDataset


class SongUserClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SongUserClassifier, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out
    

num_users = 1  # Change this to the number of users you have
user_tensors = torch.eye(num_users)  # Create one-hot encoded tensors for users
labels = [1,1,1,1,1,0,0,0,0]
embeddings = np.load("embedding.npy")

embeddings_tensor = torch.tensor(embeddings, dtype=torch.float32)
user_tensors_tensor = torch.tensor(user_tensors, dtype=torch.float32)
labels_tensor = torch.tensor(labels, dtype=torch.float32)
input_tensor = torch.cat((user_tensors_tensor.repeat(len(labels), 1), embeddings_tensor), dim=1)
print(input_tensor.shape)
input_size = input_tensor.shape[1]
hidden_size = 64
learning_rate = 0.001
batch_size = 4  # Adjust the batch size as needed
num_epochs = 100

# Create DataLoader for batch processing
dataset = TensorDataset(input_tensor, labels_tensor)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
model = SongUserClassifier(input_size, hidden_size,1)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    total_loss = 0.0
    for inputs, labels in dataloader:
        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs.squeeze(), labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item() * inputs.size(0)

    # Print average loss for the epoch
    epoch_loss = total_loss / len(dataset)
    # print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}')

# Extract learned user embeddings
user_embeddings = model.fc1.weight.data.numpy()

# Print learned user embeddings
# print("Learned User Embeddings:")
# print(user_embeddings)
def nearestNeighbour( queryEmbeddings, k):
     quantize = faiss.IndexFlatL2(queryEmbeddings.shape[1])
     index = faiss.IndexIVFFlat(quantize, queryEmbeddings.shape[1], 5)
     D, I = index.search(queryEmbeddings, k)
     print(-1)
     df = pd.read_csv("preprocessed_songs.csv")
     print(0)
     artist_songs = dict()
     allRelatable = df.iloc[I[0]]
     for index, rows in allRelatable.iterrows():
         print(1)
         print(rows["artist"])
         artist_songs[rows["artist"]] = rows["song"]
    
     print("Query: ", queryEmbeddings)
     print(artist_songs)
nearestNeighbour(user_embeddings,4)

/var/folders/_3/hpp2kxf54pndqxk7bcfbzl6w0000gn/T/ipykernel_38932/456693216.py:25: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  user_tensors_tensor = torch.tensor(user_tensors, dtype=torch.float32)
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


torch.Size([9, 385])


NameError: name 'faiss' is not defined

In [6]:

user_tensors = torch.eye(10) 
user_tensors

tensor([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]])

In [1]:
from pyechonest import catalog
cat = catalog.Catalog('CACNYVZ1332EB0BA9D')
cat.read()

TabError: inconsistent use of tabs and spaces in indentation (catalog.py, line 327)

In [9]:
import torch
import torch.nn as nn
from torch import optim
from torch.utils.data import DataLoader, TensorDataset
from torchvision import transforms

class fs_model(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(fs_model, self).__init__()
        self.relu = nn.ReLU()  
        self.sigmoid = nn.Sigmoid();
        self.fc1 = nn.Linear(input_dim, hidden_dim) 
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.fc3 = nn.Linear(hidden_dim, output_dim) 

    def forward(self, x):
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        x = self.sigmoid(x);
        return x


In [15]:
num_users = 50  # Change this to the number of users you have
user_tensors = torch.eye(num_users)  # Create one-hot encoded tensors for users
labels = [1,1,1,1,1,0,0,0,0]
embeddings = np.load("embedding.npy")

embeddings_tensor = torch.tensor(embeddings, dtype=torch.float32)
user_tensors_tensor = torch.tensor(user_tensors, dtype=torch.float32)
labels_tensor = torch.tensor(labels, dtype=torch.float32)
input_tensor = torch.cat((user_tensors_tensor.repeat(len(labels), 1), embeddings_tensor), dim=1)
input_size = input_tensor.shape[1]

model = fs_model(input_size,50,1)
loss_func  = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
num_epochs = 100

transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

dataset = TensorDataset(input_tensor, labels_tensor)
data_train, data_test = torch.utils.data.random_split(dataset, [7, 2])

train_loader = DataLoader(data_train, batch_size=32, shuffle=True)
test_loader = DataLoader(data_test, batch_size=32, shuffle=False)

for epoch in range(num_epochs):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = model(data)
        loss = loss(output, target)
        loss.backward()
        optimizer.step()
        print('Epoch {}, Batch {}, Loss {}'.format(epoch, batch_idx, loss.item()))

model.eval()
correct = 0
total = 0
with torch.no_grad():
    for data, target in test_loader:
        output = model(data)
        _, predicted = torch.max(output.data, 1)
        total += target.size(0)
        correct += (predicted == target).sum().item()

print('Accuracy on test set: {}%'.format(100*correct/total))

/var/folders/_3/hpp2kxf54pndqxk7bcfbzl6w0000gn/T/ipykernel_64492/2226937525.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  user_tensors_tensor = torch.tensor(user_tensors, dtype=torch.float32)


RuntimeError: Sizes of tensors must match except in dimension 1. Expected size 450 but got size 9 for tensor number 1 in the list.

In [12]:
import optuna
import numpy as np

import json

with open('user_0_songs.json', 'r') as file:
    data_dict = json.load(file)


num_users = 1  # Change this to the number of users you have
user_tensors = torch.eye(num_users)  # Create one-hot encoded tensors for users
dat = pd.read_csv('preprocessed_songs_gram.csv')
dat = dat['Label']
labels = dat[(dat == '0') | (dat == '1')]
labels = [int(x) for x in labels]
labels = labels[:251]
embeddings = np.load("embedding_lyrics_total.npy")

loss_func  = nn.MSELoss()

embeddings_tensor = torch.tensor(embeddings, dtype=torch.float32)
user_tensors_tensor = torch.tensor(user_tensors, dtype=torch.float32)
labels_tensor = torch.tensor(labels, dtype=torch.float32)
input_tensor = torch.cat((user_tensors_tensor.repeat(len(labels), 1), embeddings_tensor), dim=1)
input_size = input_tensor.shape[1]

def training(model, train_loader, test_loader, loss, optimizer, num_epochs):
    for epoch in range(num_epochs):
        model.train()
        for batch_idx, (data, target) in enumerate(train_loader):
            optimizer.zero_grad()
            output = model(data)
            loss = loss_func(output, target)
            loss.backward()
            optimizer.step()
            # print('Epoch {}, Batch {}, Loss {}'.format(epoch, batch_idx, loss.item()))

        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for data, target in test_loader:
                output = model(data)
                _, predicted = torch.max(output.data, 1)
                total += target.size(0)
                correct += (predicted == target).sum().item()

        # print('Accuracy on test set: {}%'.format(100*correct/total))

def objective(trial):
    hidden_dim = trial.suggest_int('hidden_dim', 50, 200)
    lr = trial.suggest_loguniform('lr', 1e-4, 1e-1)
    batch_size = trial.suggest_categorical('batch_size', [32, 64, 128])
    num_epochs = 100

    model = fs_model(input_size,50,1)
    loss_func  = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    dataset = TensorDataset(input_tensor, labels_tensor)
    data_train, data_test = torch.utils.data.random_split(dataset, [211, 40])
    train_loader = DataLoader(data_train, batch_size=32, shuffle=True)
    test_loader = DataLoader(data_test, batch_size=32, shuffle=False)

    training(model, train_loader, test_loader, loss_func, optimizer, num_epochs)

    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data, target in test_loader:
            output = model(data)
            _, predicted = torch.max(output.data, 1)
            total += target.size(0)
            correct += (predicted == target).sum().item()

    accuracy = 100 * correct / total
    return accuracy
    
    

study = optuna.create_study(direction='maximize')


study.optimize(objective, n_trials=10)


best_params = study.best_params
best_accuracy = study.best_value

print("Best parameters:", best_params)
print("Best validation accuracy:", best_accuracy)

/var/folders/_3/hpp2kxf54pndqxk7bcfbzl6w0000gn/T/ipykernel_73138/3706396032.py:20: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  user_tensors_tensor = torch.tensor(user_tensors, dtype=torch.float32)
[I 2024-05-07 11:51:17,828] A new study created in memory with name: no-name-e7139b07-e340-4863-a1f6-94131b1be2f7
/var/folders/_3/hpp2kxf54pndqxk7bcfbzl6w0000gn/T/ipykernel_73138/3706396032.py:50: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  lr = trial.suggest_loguniform('lr', 1e-4, 1e-1)
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([32])) that is different to the input size

TypeError: 'Tensor' object is not callable

In [10]:
import tkinter as tk


song_arr = []

shown = False

def song_list():
    if shown:
        shown = True
    else:
        shown = False
    
    if shown:
        song_list.config(text=f"{song_arr}!")
    else:
        song_list.config(text="")


def submit_song():
    song = entry.get()
    if song in song_arr:
        songs_2.config(text=f"{song} is already in list!")
    else:
        song_arr.append(song)
    if song:
        song_list.config(text=f"{song_arr}!")
    else:
       song_list.config(text="Input some songs")


root = tk.Tk()
root.title("Spotify Song Recommender")


entry = tk.Entry(root, width=30)
entry.pack(pady=10)

song_button = tk.Button(root, text="Submit song", command=submit_song)
song_button.pack()

list_button = tk.Button(root, text="Show song list", command=song_list)
list_button.pack()

song_list = tk.Label(root, text="")

songs = tk.Label(root, text="Songs:")
songs_2 = tk.Label(root, text="")
song_list = tk.Label(root, text="")

songs.pack()
songs_2.pack()
song_list.pack(pady=10)


root.mainloop()

Exception in Tkinter callback
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/tkinter/__init__.py", line 1921, in __call__
    return self.func(*args)
  File "/var/folders/_3/hpp2kxf54pndqxk7bcfbzl6w0000gn/T/ipykernel_69786/3149611433.py", line 9, in song_list
    if shown:
UnboundLocalError: local variable 'shown' referenced before assignment
Exception in Tkinter callback
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/tkinter/__init__.py", line 1921, in __call__
    return self.func(*args)
  File "/var/folders/_3/hpp2kxf54pndqxk7bcfbzl6w0000gn/T/ipykernel_69786/3149611433.py", line 9, in song_list
    if shown:
UnboundLocalError: local variable 'shown' referenced before assignment
Exception in Tkinter callback
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/tkinter/__init__.py", line 1921, in __call__
  

: 